In [21]:
import pandas as pd
import re

In [22]:
df = pd.read_csv("biorxiv.csv")

In [23]:
#Affiliatin_nums: Replace NAs with 1, for single affiliation.
#Affiliation_nums: Split into list
df.affiliation_nums = df.affiliation_nums.fillna('1')
df.affiliation_nums = df.affiliation_nums.apply(str.split,sep=',')

In [24]:
#Affiliation_text: Replace NAs with empty string
#Run clean_affiliation, to 
def clean_affiliation(x):
    x = x.split(';')
    for i in range(len(x)):
        x[i] = re.sub('(\\n)(\\t)+','',x[i])
        x[i] = x[i].strip()
        x[i] = re.sub('^,+','',x[i])
        x[i] = x[i].strip()
    return(x)
df.affiliation_text = df.affiliation_text.fillna('')
df.affiliation_text = df.affiliation_text.apply(clean_affiliation)

In [25]:

def clean_downloads(x):
    x = x.split(',')
    x = map(lambda y: re.sub('<tr class="odd">','',y),x)
    x = map(lambda y: re.sub('<tr class="even">','',y),x)
    x = map(lambda y: re.sub(' </tr>','',y),x)
    x = list(map(lambda y: y.split('</td>'),x))
    for i in range(len(x)):
        x[i] = list(map(lambda y: re.sub('^<td>','',y),x[i]))
    return x
df.downloads = df.downloads.fillna('')
df.downloads = df.downloads.apply(clean_downloads)

In [26]:
df

,affiliation_nums,affiliation_text,author_names,date_posted,doi,downloads,paper_title,subject_area
0,[1],[University of Southern California],"James H Joly,Brandon TL Chew,Nicholas Alexande...","\n Posted October 07, 2020.",https://doi.org/10.1101/2020.10.05.327429,"[[Oct 2020, 500, 27, 170, ]]",The landscape of metabolic pathway dependencie...,"Cancer Biology,\n"
1,"[1, 3, 4, 1, 2, 3, 2, 3, 2, 3, 2, 2, 3, 2, 3, ...","[Glasgow G61 1QH, United Kingdom,, Glasgow G61...","Mate Naszai,Yachuan Yu,Alvaro R Fernandez,Emma...","\n Posted October 08, 2020.",https://doi.org/10.1101/2020.10.07.329607,"[[Oct 2020, 343, 38, 63, ]]",RAL GTPases mediate EGFR/MAPK signalling-drive...,"Cancer Biology,\n"
2,"[1, 4, 2, 3, 3, 2, 2]",[H. Lee Moffitt Cancer Center & Research Insti...,"Jeffrey West,Maximilian Strobl,Cole Armagost,R...","\n Posted October 09, 2020.",https://doi.org/10.1101/2020.10.08.331678,"[[Oct 2020, 280, 28, 68, ]]",Antifragile therapy,"Cancer Biology,\n"
3,"[1, 10, 1, 1, 1, 1, 2, 1, 1, 1, 3, 3, 4, 4, 4,...","[University of Cambridge, Cancer Evolution and...","Filipe Correia Martins,Dominique-Laurent Coutu...","\n Posted October 12, 2020.",https://doi.org/10.1101/2020.10.04.325365,"[[Oct 2020, 768, 60, 186, ]]",Somatic chromosomal number alterations affecti...,"Cancer Biology,\n"
4,"[1, 2, 3, 1, 4, 5, 4, 6, 7, 1, 9, 8]","[Helen Diller Comprehensive Cancer Center, Uni...","Andrew L Wolfe,Qingwen Zhou,Eneda Toska,Jacque...","\n Posted October 13, 2020.",https://doi.org/10.1101/2020.10.13.337998,"[[Oct 2020, 353, 31, 66, ]]","UDP-glucose pyrophosphorylase 2, a regulator o...","Cancer Biology,\n"
...,...,...,...,...,...,...,...,...
3634,"[1, +, #, 1, 2, 3, 4, 2, 5, 1, *]","[Tucson, Arizona,, Tucson, Arizona,, Tucson, A...","Brenna A. Rheinheimer,Ronald L. Heimark,Adam D...","\n Posted September 30, 2020.",https://doi.org/10.1101/2020.09.29.318873,"[[Sep 2020, 194, 0, 18, ], [Oct 2020, 264, 38,...","Cell intrinsic signaling in , mutant pancreati...","Cancer Biology,\n"
3635,"[1, 2, 1, 1, 1, 3, 4, 1, 1, 1, 1, 1, *, 5, 6, ...",[],"Lei Huang,Xiao-Ou Zhang,Odette Verdejo-Torres,...","\n Posted September 30, 2020.",https://doi.org/10.1101/2020.08.12.246660,"[[Aug 2020, 703, 48, 198, ], [Sep 2020, 87, 14...",Protein arginine methyltransferase 5 promotes ...,"Cancer Biology,\n"
3636,"[1, 5, 2, 5, 2, 5, 2, 1, 5, 3, 1, 4, 1, 5, *]","[Houston TX,, Houston TX,, Houston TX,, Housto...","Emilly S Villodre,Yun Gong,Lei Huo,Esther C Yo...","\n Posted September 28, 2020.",https://doi.org/10.1101/2020.09.25.313817,"[[Sep 2020, 372, 2, 38, ], [Oct 2020, 157, 28,...",NDRG1 expression is an independent prognostic ...,"Cancer Biology,\n"
3637,"[1, #, 1, #, 2, 1, 3, 3, 3, 3, 3, *, 1, *]","[Beijing 100037, China,, Chengdu 610041, China...","Xuanyu Liu,Meng Yuan,Qinqin Xiang,Wen Chen,Zhu...","\n Posted September 28, 2020.",https://doi.org/10.1101/2020.09.27.315911,"[[Sep 2020, 248, 2, 45, ], [Oct 2020, 160, 60,...",Single-cell RNA-seq of the stromal vascular fr...,"Cancer Biology,\n"


In [44]:
from pymed import PubMed

def get_pmid(authors):
    authors = authors.split(',')
    if (len(authors)<3):
        return "Too few"
    query = f"((({authors[0]}[Author]) AND ({authors[1]}[Author])) AND ({authors[2]}[Author])"
    pubmed = PubMed(tool="gneng")
    #query ="(((Constantin Georgescu[Author]) AND (Joshua M. Corbin[Author])) AND (Jonathan D. Wren[Author])) AND (Maria J. Ruiz-Echevarría[Author])"
    pubmed_results = pubmed.query(query, max_results=10)
    pmid_list = []
    for article in pubmed_results:
        pubmed_id = article.toJSON()
        pubmed_id = pubmed_id.split("\"pubmed_id\": \"")
        pubmed_id = pubmed_id[1].split("\\n")
        pmid_list.append(pubmed_id[0])
    return pmid_list

In [45]:
test = df.author_names.head(10).apply(get_pmid)

HTTPError: 429 Client Error: Too Many Requests for url: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?tool=gneng&email=my_email%40example.com&db=pubmed&term=%28%28%28Filipe+Correia+Martins%5BAuthor%5D%29+AND+%28Dominique-Laurent+Couturier%5BAuthor%5D%29%29+AND+%28Ines+Santiago%5BAuthor%5D%29&retmax=10&retmode=json

In [41]:
get_pmid(df.author_names[0])

[]

In [42]:
df.author_names[0]

'James H Joly,Brandon TL Chew,Nicholas Alexander Graham'

In [ ]:
((James H Joly[Author]) AND (Brandon TL Chew[Author])) AND (Nicholas Alexander Graham[Author])

In [8]:
query ="(((Constantin Georgescu[Author]) AND (Joshua M. Corbin[Author])) AND (Jonathan D. Wren[Author])) AND (Maria J. Ruiz-Echevarría[Author])"


In [9]:
query

'(((Constantin Georgescu[Author]) AND (Joshua M. Corbin[Author])) AND (Jonathan D. Wren[Author])) AND (Maria J. Ruiz-Echevarría[Author])'

In [15]:
pubmed_article.split("\"pubmed_id\":")

['{\n    "abstract": "The clinical behavior of prostate cancer (PCa) is variable, and while the majority of cases remain indolent, 10% of patients progress to deadly forms of the disease. Current clinical predictors used at the time of diagnosis have limitations to accurately establish progression risk. Here we describe the development of a tumor suppressor regulated, cell-cycle gene expression based prognostic signature for PCa, and validate its independent contribution to risk stratification in several radical prostatectomy (RP) patient cohorts.\\nWe used RNA interference experiments in PCa cell lines to identify a gene expression based gene signature associated with Tmeff2, an androgen regulated, tumor suppressor gene whose expression shows remarkable heterogeneity in PCa. Gene expression was confirmed by qRT-PCR. Correlation of the signature with disease outcome (time to recurrence) was retrospectively evaluated in four geographically different cohorts of patients that underwent RP

In [46]:
df

,affiliation_nums,affiliation_text,author_names,date_posted,doi,downloads,paper_title,subject_area
0,[1],[University of Southern California],"James H Joly,Brandon TL Chew,Nicholas Alexande...","\n Posted October 07, 2020.",https://doi.org/10.1101/2020.10.05.327429,"[[Oct 2020, 500, 27, 170, ]]",The landscape of metabolic pathway dependencie...,"Cancer Biology,\n"
1,"[1, 3, 4, 1, 2, 3, 2, 3, 2, 3, 2, 2, 3, 2, 3, ...","[Glasgow G61 1QH, United Kingdom,, Glasgow G61...","Mate Naszai,Yachuan Yu,Alvaro R Fernandez,Emma...","\n Posted October 08, 2020.",https://doi.org/10.1101/2020.10.07.329607,"[[Oct 2020, 343, 38, 63, ]]",RAL GTPases mediate EGFR/MAPK signalling-drive...,"Cancer Biology,\n"
2,"[1, 4, 2, 3, 3, 2, 2]",[H. Lee Moffitt Cancer Center & Research Insti...,"Jeffrey West,Maximilian Strobl,Cole Armagost,R...","\n Posted October 09, 2020.",https://doi.org/10.1101/2020.10.08.331678,"[[Oct 2020, 280, 28, 68, ]]",Antifragile therapy,"Cancer Biology,\n"
3,"[1, 10, 1, 1, 1, 1, 2, 1, 1, 1, 3, 3, 4, 4, 4,...","[University of Cambridge, Cancer Evolution and...","Filipe Correia Martins,Dominique-Laurent Coutu...","\n Posted October 12, 2020.",https://doi.org/10.1101/2020.10.04.325365,"[[Oct 2020, 768, 60, 186, ]]",Somatic chromosomal number alterations affecti...,"Cancer Biology,\n"
4,"[1, 2, 3, 1, 4, 5, 4, 6, 7, 1, 9, 8]","[Helen Diller Comprehensive Cancer Center, Uni...","Andrew L Wolfe,Qingwen Zhou,Eneda Toska,Jacque...","\n Posted October 13, 2020.",https://doi.org/10.1101/2020.10.13.337998,"[[Oct 2020, 353, 31, 66, ]]","UDP-glucose pyrophosphorylase 2, a regulator o...","Cancer Biology,\n"
...,...,...,...,...,...,...,...,...
3634,"[1, +, #, 1, 2, 3, 4, 2, 5, 1, *]","[Tucson, Arizona,, Tucson, Arizona,, Tucson, A...","Brenna A. Rheinheimer,Ronald L. Heimark,Adam D...","\n Posted September 30, 2020.",https://doi.org/10.1101/2020.09.29.318873,"[[Sep 2020, 194, 0, 18, ], [Oct 2020, 264, 38,...","Cell intrinsic signaling in , mutant pancreati...","Cancer Biology,\n"
3635,"[1, 2, 1, 1, 1, 3, 4, 1, 1, 1, 1, 1, *, 5, 6, ...",[],"Lei Huang,Xiao-Ou Zhang,Odette Verdejo-Torres,...","\n Posted September 30, 2020.",https://doi.org/10.1101/2020.08.12.246660,"[[Aug 2020, 703, 48, 198, ], [Sep 2020, 87, 14...",Protein arginine methyltransferase 5 promotes ...,"Cancer Biology,\n"
3636,"[1, 5, 2, 5, 2, 5, 2, 1, 5, 3, 1, 4, 1, 5, *]","[Houston TX,, Houston TX,, Houston TX,, Housto...","Emilly S Villodre,Yun Gong,Lei Huo,Esther C Yo...","\n Posted September 28, 2020.",https://doi.org/10.1101/2020.09.25.313817,"[[Sep 2020, 372, 2, 38, ], [Oct 2020, 157, 28,...",NDRG1 expression is an independent prognostic ...,"Cancer Biology,\n"
3637,"[1, #, 1, #, 2, 1, 3, 3, 3, 3, 3, *, 1, *]","[Beijing 100037, China,, Chengdu 610041, China...","Xuanyu Liu,Meng Yuan,Qinqin Xiang,Wen Chen,Zhu...","\n Posted September 28, 2020.",https://doi.org/10.1101/2020.09.27.315911,"[[Sep 2020, 248, 2, 45, ], [Oct 2020, 160, 60,...",Single-cell RNA-seq of the stromal vascular fr...,"Cancer Biology,\n"


In [56]:
df.iloc[2200].author_names

'Jong Wook Kim,Christian Berrios,Miju Kim,Amy E. Schade,Guillaume Adelmant,Huwate Yeerna,Emily Damato,Amanda Balboni Iniguez,Selene K. Swanson,Laurence Florens,Michael P. Washburn,Kim Stegmaier,Nathaniel S. Gray,Pablo Tamayo,Ole Gjoerup,Jarrod A. Marto,James A. DeCaprio,William C. Hahn'

In [71]:
df.author_names.head(100).apply(get_pubmed_id).values

array(['https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=author&term=james+h+joly+AND+brandon+tl+chew+AND+nicholas+alexander+graham&api_key=56a5fd6220202c3f7c1c924f215c4188b708',
       'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=author&term=mate+naszai+AND+yachuan+yu+AND+alvaro+r+fernandez&api_key=56a5fd6220202c3f7c1c924f215c4188b708',
       'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=author&term=jeffrey+west+AND+maximilian+strobl+AND+cole+armagost&api_key=56a5fd6220202c3f7c1c924f215c4188b708',
       'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=author&term=filipe+correia+martins+AND+dominique-laurent+couturier+AND+ines+santiago&api_key=56a5fd6220202c3f7c1c924f215c4188b708',
       'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=author&term=andrew+l+wolfe+AND+qingwen+zhou+AND+eneda+toska&api_key=56a5fd6220202c3f7c1c924f215c4188b708',
   

In [78]:
import requests
import re
import time

In [102]:
def get_pubmed_id(authors):
    base = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
    esearch= 'esearch.fcgi?'
    db= 'db=pubmed&'
    field= 'field=author&'
    term_prefix = 'term='
    and_syntax = '+AND+'
    term_suffix = '&'
    api = 'api_key=56a5fd6220202c3f7c1c924f215c4188b708'
    
    if authors=='':
        return "No authors"
    authors = authors.lower()
    authors = authors.split(',')
    
    if len(authors) > 2:
        term = re.sub(' ','+',authors[0])+and_syntax+re.sub(' ','+',authors[1])+and_syntax+re.sub(' ','+',authors[2])
    elif len(authors) > 1:
        term = re.sub(' ','+',authors[0])+and_syntax+re.sub(' ','+',authors[1])
    else:
        term = re.sub(' ','+',authors[0])
        
    url = base+esearch+db+field+term_prefix+term+term_suffix+api
    
    response=requests.get(url)
    
    return response.text

In [104]:
author_list = list(df.author_names.fillna('').values)

In [103]:
len(df.author_names.fillna('').values)

3639

In [105]:
len(author_list)

3639

In [106]:
pmid_result = []
for i in range(len(author_list)):
    tmp = get_pubmed_id(author_list[i])
    pmid_result.append(tmp)
    time.sleep(0.2)


In [110]:
df2 = pd.Series(pmid_result)

In [116]:
df2.to_csv("test.csv")